# Object-oriented FaIR

Why do classes make absolutely zero sense?

**TODO** sort out the fact that lifetime adjustments seem to be relative to present day for CO2 and pre-industrial for CH4

In [ ]:
from abc import ABC
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as pl

from fair21.constants.gases import MOLWT
from fair21.constants import M_ATMOS, TIME_AXIS, SPECIES_AXIS
from fair21.defaults.gases import iirf_horizon, iirf_max, pre_industrial_concentration
from fair21.defaults import f_gas_list, montreal_gas_list, slcf_list, gas_list, n_gas_boxes, n_temperature_boxes, species_list
from fair21.energy_balance_model import EnergyBalanceModel
from fair21.exceptions import MissingInputError, IncompatibleConfigError, PartitionFractionError, LifetimeError, TimeNotDefinedError
from fair21.gas_cycle import calculate_alpha
from fair21.gas_cycle.forward import step_concentration
from fair21.forcing.ghg import ghg

In [ ]:
scenarios = ['ssp119', 'ssp126', 'ssp245', 'ssp370', 'ssp434', 'ssp460', 'ssp534-over', 'ssp585']

In [ ]:
# grab some emissions
emissions = {}

df = pd.read_csv('../data/rcmip/rcmip-emissions-annual-means-v5-1-0.csv')
for iscen, scenario in enumerate(scenarios):
    emissions[scenario] = {}
    for ispec, specie in enumerate(species_list):
        specie_rcmip_name = specie.replace("-", "")
        emissions[scenario][specie] = df.loc[
            (df['Scenario']==scenario) & (df['Variable'].str.endswith("|"+specie_rcmip_name)) & (df['Region']=='World'), '1750':
        ].interpolate(axis=1).values.T.squeeze()
    
        # CO2 and N2O units need to behave
        if specie in ('CO2', 'N2O'):
            emissions[scenario][specie] = emissions[scenario][specie] / 1000

In [ ]:
scenario = 'ssp245'

`Specie` is an abstract base class. Everything that we care about for climate is a `Specie`, but we should encourage users to use derived classes like `GreenhouseGas`

In [ ]:
class Specie(ABC):
    
    def __init__(self, name, tropospheric_adjustment=0):
        self.name = name
        self.tropospheric_adjustment=tropospheric_adjustment
    
    def set_index(self, index):
        self.index = index

In [ ]:
class GreenhouseGas(Specie):
    
    def __init__(self, name, molecular_weight, lifetime, radiative_efficiency, **kwargs):
        tropospheric_adjustment = kwargs.pop('tropospheric_adjustment', 0)
        super().__init__(name, tropospheric_adjustment)
        
        # move the below to input verification method
        self.molecular_weight = molecular_weight
        self.lifetime = lifetime
        self.radiative_efficiency = radiative_efficiency
        if np.ndim(self.lifetime) == 1:
            lifetime = np.asarray(lifetime)
            # should we enforce whether strictly decreasing or not?
            partition_fraction = kwargs.get('partition_fraction')
            if partition_fraction is None:
                raise MissingInputError('specify `partition_fraction` if specifying more than one `lifetime`') # custom exception needed
            if len(partition_fraction) != len(lifetime):
                raise IncompatibleConfigError('`partition_fraction` and `lifetime` are different shapes') # custom exception needed
            partition_fraction = np.asarray(partition_fraction)
            if ~np.isclose(np.sum(partition_fraction), 1):
                raise PartitionFractionError('partition_fraction should sum to 1') # custom exception needed
        elif np.ndim(self.lifetime) > 1:
            raise LifetimeError('`lifetime` array dimension is greater than 1')
        else:
            partition_fraction=np.zeros(n_gas_boxes)
            partition_fraction[0] = 1
        self.partition_fraction=partition_fraction
        self.emissions = kwargs.get('emissions')
        self.concentration = kwargs.get('concentration')
        self.forcing = kwargs.get('forcing')
        self.g0, self.g1 = self.calculate_g(partition_fraction, lifetime, iirf_horizon=iirf_horizon)
        self.iirf_0 = kwargs.get('iirf_0', self.lifetime_to_iirf_0(lifetime, partition_fraction, iirf_horizon=iirf_horizon))
        self.iirf_cumulative = kwargs.get('iirf_cumulative', 0)
        self.iirf_airborne = kwargs.get('iirf_airborne', 0)
        self.iirf_temperature = kwargs.get('iirf_temperature', 0)
        self.pre_industrial_concentration=kwargs.get('pre_industrial_concentration', 0)
        self.natural_emissions_adjustment=kwargs.get('natural_emissions_adjustment', 0)
        self.output_forcing=True


    
    @staticmethod
    def calculate_g(partition_fraction, lifetime, iirf_horizon=100):
        g1 = np.sum(partition_fraction * lifetime * (1 - (1 + iirf_horizon/lifetime) * np.exp(-iirf_horizon/lifetime)))
        g0 = np.exp(-1 * np.sum(partition_fraction*lifetime*(1 - np.exp(-iirf_horizon/lifetime)))/g1)
        return g0, g1
    
    @staticmethod
    def lifetime_to_iirf_0(lifetime, partition_fraction=1,iirf_horizon=100):
        return np.sum(lifetime * (1 - np.exp(-iirf_horizon / lifetime)) * partition_fraction)
    
    @property
    def burden_per_emission(self):
        return 1 / (M_ATMOS / 1e18 * self.molecular_weight / MOLWT['AIR'])

In [ ]:
class CO2(GreenhouseGas):
    def __init__(self, **kwargs):
        self.molecular_weight=kwargs.pop('molecular_weight', 44.009)
        self.lifetime = kwargs.pop('lifetime', np.array([1e9, 394.4, 36.54, 4.304]))
        self.partition_fraction = kwargs.pop('partition_fraction', np.array([0.2173, 0.2240, 0.2824, 0.2763]))
        self.tropospheric_adjustment = kwargs.pop('tropospheric_adjustment', 0.05)
        self.radiative_efficiency = kwargs.pop('radiative_efficiency', 1.3344985680386619e-05)
        # we should calculate based on Meinshausen formula and provide this as a convenience method
        # allowing for time variation
        
        super().__init__(
            name="CO2",
            molecular_weight=self.molecular_weight, 
            lifetime=self.lifetime,
            partition_fraction=self.partition_fraction,
            tropospheric_adjustment=self.tropospheric_adjustment,
            radiative_efficiency=self.radiative_efficiency,
            **kwargs
        )
        self.emissions_unit = kwargs.get('emissions_unit', 'Gt CO2/yr')
        self.concentration_unit = kwargs.get('concentration_unit', 'ppm')
        self.iirf_0 = kwargs.get('iirf_0', 29)
        self.iirf_cumulative = kwargs.get('iirf_cumulative', 0.00846)
        self.iirf_airborne = kwargs.get('iirf_airborne', 0.000819)
        self.iirf_temperature = kwargs.get('iirf_temperature', 4)
        self.pre_industrial_concentration=kwargs.get('pre_industrial_concentration', 278.3)
        
        
    # needs CH4 and N2O concentration
    def calculate_radiative_efficiency():
        pass

In [ ]:
co2 = CO2()

In [ ]:
vars(co2)

In [ ]:
co2.burden_per_emission

In [ ]:
class Halogen(GreenhouseGas):
    def __init__(self, cl_atoms, br_atoms, fractional_release, **kwargs):
        super().__init__(**kwargs)
        self.cl_atoms = cl_atoms
        self.br_atoms = br_atoms
        self.fractional_release = fractional_release
        #self.radiative_efficiency = kwargs.get('radiative_efficiency')
        self.ozone_radiative_efficiency = -1.25e-4
        self.ozone_forcing_based_on = 'concentration'
        self.emissions_unit = 'kt {}/yr'.format(self.name.replace("-", ""))
        self.concentration_unit = 'ppt'
        self.pre_industrial_concentration=kwargs.get('pre_industrial_concentration', 0)
        
#     def calculate_eesc():
#         pass
    
#         eesc = calculate_eesc(
#         concentration,
#         pre_industrial_concentration,
#         fractional_release,
#         cl_atoms,
#         br_atoms,
#         species_index_mapping,
#         br_cl_ratio=br_cl_ratio,
#     )

In [ ]:
# archetype for default
cfc_11 = Halogen(name='CFC-11', tropospheric_adjustment=0.13, lifetime=45, fractional_release=0.23, cl_atoms=2, br_atoms=0, molecular_weight=500, radiative_efficiency=0.2)

In [ ]:
vars(cfc_11)

In [ ]:
class FGas(GreenhouseGas):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.emissions_unit = 'kt {}/yr'.format(self.name.replace("-", ""))
        self.concentration_unit = 'ppt'
        self.pre_industrial_concentration=0

In [ ]:
hfc134a = FGas(name='HFC-134a', lifetime=23, radiative_efficiency=0.3, molecular_weight=140)

In [ ]:
vars(hfc134a)

In [ ]:
class CH4(GreenhouseGas):
    def __init__(self, **kwargs):
        self.molecular_weight=kwargs.pop('molecular_weight', 16.043)
        self.lifetime = kwargs.pop('lifetime', 8.25)
        self.tropospheric_adjustment = kwargs.pop('tropospheric_adjustment', -0.14)
        self.radiative_efficiency = kwargs.pop('radiative_efficiency', 0.00038864402860869495) # again this calcu needs to be done specific
        
        super().__init__(
            name='CH4',
            molecular_weight=self.molecular_weight, 
            lifetime=self.lifetime,
            tropospheric_adjustment=self.tropospheric_adjustment,
            radiative_efficiency=self.radiative_efficiency,
            **kwargs
        )
        self.ozone_radiative_efficiency = 1.75e-4
        self.ozone_forcing_based_on = 'concentration'
        self.emissions_unit = kwargs.get('emissions_unit', 'Mt CH4/yr')
        self.concentration_unit = kwargs.get('concentration_unit', 'ppb')
        self.iirf_0 = kwargs.get('iirf_0')
        self.iirf_airborne = kwargs.get('iirf_airborne', 0.00032)
        self.iirf_temperature = kwargs.get('iirf_temperature', -0.3)
        self.pre_industrial_concentration=kwargs.get('pre_industrial_concentration', 729.2)

In [ ]:
ch4 = CH4()

In [ ]:
vars(ch4)

In [ ]:
class N2O(GreenhouseGas):
    def __init__(self, **kwargs):
        self.molecular_weight=kwargs.pop('molecular_weight', 44.013)
        self.lifetime = kwargs.pop('lifetime', 109)
        self.tropospheric_adjustment = kwargs.pop('tropospheric_adjustment', 0.07)
        self.radiative_efficiency = kwargs.pop('radiative_efficiency', 0.00319550741640458) # again this calcu needs to be done specific

        super().__init__(
            name='N2O',
            molecular_weight=self.molecular_weight, 
            lifetime=self.lifetime,
            tropospheric_adjustment=self.tropospheric_adjustment,
            radiative_efficiency=self.radiative_efficiency,
            **kwargs
        )
        self.ozone_radiative_efficiency = 7.10e-4
        self.ozone_forcing_based_on = 'concentration'
        self.emissions_unit = kwargs.get('emissions_unit', 'Mt N2O/yr')
        self.concentration_unit = kwargs.get('concentration_unit', 'ppb')
        self.iirf_0 = kwargs.get('iirf_0')
        self.pre_industrial_concentration = kwargs.get('pre_industrial_concentration', 270.1)

In [ ]:
n2o = N2O()
vars(n2o)

In [ ]:
# now let's say we want to simulate UKESM and say CH4 is indeed an aerosol precursor. Can we do this?
# it doesn't complain...
ch4 = CH4(ari_radiative_efficiency=0.4)

In [ ]:
#...but the argument is unused and doesnt appear as instance of CH4. This needs to be sorted out
vars(ch4)

In [ ]:
class ShortLivedForcer(Specie):
    
    def __init__(self, name, **kwargs):
        super().__init__(name, **kwargs)
        self.emissions = kwargs.get('emissions')
        self.forcing = kwargs.get('forcing')
        self.emissions_unit = kwargs.get('emissions_unit', 'Mt {}/yr'.format(self.name.replace("-", "")))
        self.pre_industrial_emissions = kwargs.get('pre_industrial_emissions', 0)

In [ ]:
# we'll make ERFari a direct property of the species, and ERFaci is a class

class AerosolPrecursor(ShortLivedForcer):
    def __init__(self, ari_radiative_efficiency, **kwargs):
        super().__init__(**kwargs)
        self.output_forcing = True

In [ ]:
class Sulfur(AerosolPrecursor):
    def __init__(self, **kwargs):
        self.ari_radiative_efficiency = kwargs.pop('ari_radiative_efficiency', -0.0036167830509091486)
        super().__init__(
            name='Sulfur',
            ari_radiative_efficiency=self.ari_radiative_efficiency,
        )
        self.emissions_unit = kwargs.get('emissions_unit', 'Mt SO2/yr')

In [ ]:
sulfur = Sulfur()
vars(sulfur)

In [ ]:
class BC(AerosolPrecursor):
    def __init__(self, **kwargs):
        self.ari_radiative_efficiency = kwargs.pop('ari_radiative_efficiency', 0.0507748226795483)
        super().__init__(
            name='BC',
            ari_radiative_efficiency=self.ari_radiative_efficiency,
            **kwargs
        )

In [ ]:
class OC(AerosolPrecursor):
    def __init__(self, **kwargs):
        self.ari_radiative_efficiency = kwargs.pop('ari_radiative_efficiency', -0.006214374446217472)
        super().__init__(
            name='OC',
            ari_radiative_efficiency=self.ari_radiative_efficiency,
            **kwargs
        )

In [ ]:
bc = BC()
vars(bc)

In [ ]:
oc = OC()
vars(oc)

In [ ]:
class Ozone(Specie):
    def __init__(self, **kwargs):
        super().__init__(name='Ozone')
        self.forcing_output=True
        self.temperature_feedback = -0.037

In [ ]:
ozone = Ozone()
vars(ozone)

In [ ]:
class AerosolCloudInteractions():
    def __init__(self, **kwargs):
        super().__init__(name='Aerosol-cloud interactions')
        self.forcing_output=True
        #scale =
        #shape_sulfur = 
        #shape_bcoc = 

In [ ]:
#species_list = [co2, ch4, n2o]

In [ ]:
run_mode = {
    'forcing driven' : False,
    'CO2' : 'emissions',
    'CH4' : 'emissions',
    'N2O' : 'emissions',
    'F-Gases': None,#'emissions',
    'Montreal Gases': None,#'emissions',
    'Aerosol-radiation interactions' : None,#'emissions',
    'Aerosol-cloud interactions' : None,#'emissions',
    'Ozone' : None,#'emissions',
    'Land use' : None, #'emissions',
    'Stratospheric water vapour':  None,#'emissions',
    'Contrails' : None,#'emissions',
    'Black carbon on snow' : None,#'emissions',
    'Solar': None,#'forcing',
    'Volcanic': None,#'forcing'
}

valid_run_modes = {
    'forcing driven' : (False, True),
    'CO2' : ('emissions', 'concentration', 'forcing', None),
    'CH4' : ('emissions', 'concentration', 'forcing', None),
    'N2O' : ('emissions', 'concentration', 'forcing', None),
    'F-Gases' : ('emissions', 'concentration', 'forcing', None),
    'Montreal Gases' : ('emissions', 'concentration', 'forcing', None),
    'Aerosol-radiation interactions' : ('emissions', 'forcing', None),
    'Aerosol-cloud interactions': ('emissions', 'forcing', None),
    'Ozone' : ('emissions', 'forcing', None),
    'Land use' : ('emissions', 'forcing', None),
    'Stratospheric water vapour' : ('emissions', 'forcing', None),
    'Contrails' : ('emissions', 'forcing', None),
    'Black carbon on snow' : ('emissions', 'forcing', None),
    'Solar' : ('forcing', None),
    'Volcanic' : ('forcing', None),
}

In [ ]:
required_species_emissions_or_concentration = {
    'forcing driven' : None,
    'CO2' : ['CO2', 'N2O'],
    'CH4' : ['CH4', 'N2O'],
    'N2O' : ['CO2', 'CH4', 'N2O'],
    'F-Gases' : None,
    'Montreal Gases' : None,
    'Aerosol-radiation interactions': None,
    'Aerosol-cloud interactions' : ['Sulfur', 'BC', 'OC'],
    'Ozone' : ['CFC-11'],
    'Land use' : ['CO2|AFOLU'],
    'Stratospheric water vapour' : ['CH4'],
    'Contrails' : ['NOx|Aviation'],
    'Black carbon on snow' : ['BC'],
    'Solar' : ['Solar'],
    'Volcanic' : ['Volcanic'],
}

required_species = []

for mode in run_mode:
    if required_species_emissions_or_concentration[mode] is None:
        continue
    if run_mode[mode] in ('emissions', 'concentration') :
        required_species.extend(required_species_emissions_or_concentration[mode])
    if mode in ('Solar', 'Volcanic'):
        if run_mode[mode]=='forcing':
            required_species.extend(required_species_emissions_or_concentration[mode])

required_species = list(set(required_species))
required_species

In [ ]:
# # check for duplicates
# species_names = []
# for specie in species_list:
#     if specie.name in species_names:
#         raise DuplicatedSpeciesError(f'{specie.name} is non-unique in the scenario')
#     species_names.append(specie.name)

# # check everything required is defined in the scenario
# for specie_name in required_species:
#     if specie_name not in species_names:
#         raise MissingInputError(f'{specie_name} is required but not in scenario')

In [ ]:
#species_names

In [ ]:
# ozone_emissions_indices = []
# ozone_concentration_indices = []
# ari_indices = []
# ghg_indices = []
# index_mapping = {}

# for ispec, specie in enumerate(species_list):
#     index_mapping[specie.name] = ispec
#     specie.set_index(ispec)
#     if hasattr(specie, "ozone_radiative_efficiency") and isinstance(specie, ShortLivedForcer):
#         ozone_emissions_indices.append(ispec)
#     if hasattr(specie, "ozone_radiative_efficiency") and isinstance(specie, GreenhouseGas):
#         ozone_concentration_indices.append(ispec)
#     if isinstance(specie, GreenhouseGas):
#         ghg_indices.append(ispec)
#     if isinstance(specie, AerosolPrecursor):
#         ari_indices.append(ispec)

In [ ]:
#ozone_emissions_indices

In [ ]:
#ozone_concentration_indices

In [ ]:
#ghg_indices

In [ ]:
#ari_indices

In [ ]:
if run_mode['Aerosol-cloud interactions']=='emissions':
    aci_indices = [index_mapping[specie] for specie in required_species_emissions_or_concentration['Aerosol-cloud interactions']] # in that order
    # then we can choose different treatments in  required_species_emissions_or_concentration['Aerosol-cloud interactions'], e.g. stevens
else:
    aci_indices = []

In [ ]:
aci_indices

In [ ]:
class Scenario():
    """The Scenario class gathers all of the Species"""

In [ ]:
class Config():
    """The Config class contains all of the parameters to run the model with"""
    
    def __init__(self, **kwargs):
        
        # Energy balance model
        self.ocean_heat_capacity = kwargs.pop('ocean_heat_capacity', np.array([5, 20, 100]))
        self.ocean_heat_transfer = kwargs.pop('ocean_heat_transfer', np.array([1, 2, 1]))
        self.deep_ocean_efficacy = kwargs.pop('deep_ocean_efficacy', 1)
        self.stochastic_run = kwargs.pop('stochastic_run', False)
        self.sigma_eta = kwargs.pop('sigma_eta', 0.5)
        self.sigma_xi = kwargs.pop('sigma_xi', 0.5)
        self.gamma_autocorrelation = kwargs.pop('gamma_autocorrelation', 2)
        self.seed = kwargs.pop('seed', None)
        
        # forcing uncertainty
        
        # gas cycle uncertainty: allow overwriting of what's in the Species
        
        # is volcanic and solar forcing a scenario or a config? think it's a scenario - don't include

In [ ]:
df = pd.read_csv(
    os.path.join("..", "data", "calibration", "4xCO2_cummins.csv")
)
models = df['model'].unique()
params = {}
for model in models:
    params[model] = {}
    for run in df.loc[df['model']==model, 'run']:
        condition = (df['model']==model) & (df['run']==run)
        params[model][run] = {}
        params[model][run]['gamma_autocorrelation'] = df.loc[condition, 'gamma'].values[0]
        params[model][run]['ocean_heat_capacity'] = df.loc[condition, 'C1':'C3'].values.squeeze()
        params[model][run]['ocean_heat_transfer'] = df.loc[condition, 'kappa1':'kappa3'].values.squeeze()
        params[model][run]['deep_ocean_efficacy'] = df.loc[condition, 'epsilon'].values[0]
        params[model][run]['sigma_eta'] = df.loc[condition, 'sigma_eta'].values[0]
        params[model][run]['sigma_xi'] = df.loc[condition, 'sigma_xi'].values[0]
        params[model][run]['forcing_4co2'] = df.loc[condition, 'F_4xCO2'].values[0]

In [ ]:
params['CanESM5']['r1i1p1f1']

In [ ]:
canesm5 = Config(**params['CanESM5']['r1i1p1f1'])
noresm2 = Config(**params['NorESM2-LM']['r1i1p1f1'])
giss = Config(**params['GISS-E2-1-G']['r1i1p1f1'])
miroc6 = Config(**params['MIROC6']['r1i1p1f1'])
hadgem3 = Config(**params['HadGEM3-GC31-LL']['r1i1p1f3'])

vars(canesm5)

In [ ]:
def _make_time_deltas(time):
    time_inner_bounds = 0.5*(time[1:] + time[:-1])
    time_lower_bound = time[0] - (time_inner_bounds[0] - time[0])
    time_upper_bound = time[-1] + (time[-1] - time_inner_bounds[-1])
    time_bounds = np.concatenate(([time_lower_bound], time_inner_bounds, [time_upper_bound]))
    time_deltas = np.diff(time_bounds)
    return time_deltas

In [ ]:
%load_ext line_profiler

In [ ]:
class FAIR():
    
    # species_list is a list of objects
    # species_names is a list of strings
        
    def __init__(self, **kwargs):
        # This is gonna be one hell of a docstring
        
        # Forcing species
        if 'species' in kwargs:
            species = kwargs.pop('species')
            self.species = {}
            if hasattr(species, '__iter__'):
                # this will automatically update/avoid duplicates
                self.species = {specie.name: specie for specie in species}
            elif species is not None:
                self.species = {species.name: species}

        for attr in kwargs:
            setattr(self, attr, kwargs[attr])
            
    def add_configs(self):
        pass
    
    
    def add_scenarios(self):
        pass
    
            
    def _assign_indices(self):
        """Assign a unique index to each specie included in the scenario."""
        self.species_index_mapping = {}
        self.ghg_indices = []
        for ispec, specie in enumerate(self.species):
            self.species_index_mapping[specie] = ispec
            if isinstance(self.species[specie], GreenhouseGas):
                self.ghg_indices.append(ispec)
            

    def _initialise_arrays(self, n_timesteps, n_scenarios, n_configs, n_species):
        self.emissions_array = np.ones((n_timesteps, n_scenarios, n_configs, n_species, 1)) * np.nan
        self.concentration_array = np.ones((n_timesteps, n_scenarios, n_configs, n_species, 1)) * np.nan
        self.forcing_sum_array = np.ones((n_timesteps, n_scenarios, n_configs, 1, 1)) * np.nan
        self.forcing_array = np.ones((n_timesteps, n_scenarios, n_configs, n_species, 1)) * np.nan
        self.g0_array = np.ones((1, n_scenarios, n_configs, n_species, 1)) * np.nan
        self.g1_array = np.ones((1, n_scenarios, n_configs, n_species, 1)) * np.nan
        self.alpha_lifetime = np.ones((1, n_scenarios, n_configs, n_species, 1))
        self.airborne_emissions = np.zeros((1, n_scenarios, n_configs, n_species, 1))
        self.gas_boxes = np.zeros((1, n_scenarios, n_configs, n_species, n_gas_boxes))
        self.iirf_0_array = np.ones((1, 1, n_configs, n_species, 1)) * np.nan
        self.iirf_cumulative_array = np.ones((1, 1, n_configs, n_species, 1)) * np.nan
        self.iirf_temperature_array = np.ones((1, 1, n_configs, n_species, 1)) * np.nan
        self.iirf_airborne_array = np.ones((1, 1, n_configs, n_species, 1)) * np.nan
        self.burden_per_emission_array = np.ones((1, 1, 1, n_species, 1)) * np.nan
        self.lifetime_array = np.ones((1, 1, n_configs, n_species, n_gas_boxes)) * np.nan
        self.pre_industrial_emissions_array = np.ones((1, n_scenarios, n_configs, n_species, 1)) * np.nan
        self.pre_industrial_concentration_array = np.ones((1, n_scenarios, n_configs, n_species, 1)) * np.nan
        self.partition_fraction_array = np.zeros((1, 1, n_configs, n_species, n_gas_boxes))
        self.natural_emissions_adjustment_array = np.zeros((1, n_scenarios, n_configs, n_species, 1))
        self.radiative_efficiency_array = np.ones((1, 1, n_configs, n_species, 1)) * np.nan
        self.tropospheric_adjustment_array = np.ones((1, 1, n_configs, n_species, 1)) * np.nan
        
        # This is an ugly amount of repetition, but I don't think it's possible to setattr on a slice of
        # a numpy array without using eval, which is even uglier
        
        # time, scenario, config, species, box
        # for iscen, scenario in enumerate(scenarios):
        # for iconfig, config in enumerate(config):
        iscen=0
        iconfig=0
        for ispec, specie in enumerate(self.species):
            if hasattr(self.species[specie], 'emissions'):
                self.emissions_array[:, iscen, iconfig, ispec, 0] = self.species[specie].emissions
            if hasattr(self.species[specie], 'concentration'):
                self.concentration_array[:, iscen, iconfig, ispec, 0] = self.species[specie].concentration
            if hasattr(self.species[specie], 'forcing'):
                self.tropospheric_adjustment_array[:, iscen, iconfig, ispec, 0] = self.species[specie].tropospheric_adjustment
                self.forcing_array[:, iscen, iconfig, ispec, 0] = self.species[specie].forcing
            if isinstance(self.species[specie], GreenhouseGas):
                self.lifetime_array[:, iscen, iconfig, ispec, :] = self.species[specie].lifetime
                self.partition_fraction_array[:, iscen, iconfig, ispec, :] = self.species[specie].partition_fraction
                self.iirf_0_array[:, iscen, iconfig, ispec, :] = self.species[specie].iirf_0
                self.iirf_cumulative_array[:, iscen, iconfig, ispec, :] = self.species[specie].iirf_cumulative
                self.iirf_temperature_array[:, iscen, iconfig, ispec, :] = self.species[specie].iirf_temperature
                self.iirf_airborne_array[:, iscen, iconfig, ispec, :] = self.species[specie].iirf_airborne
                self.burden_per_emission_array[:, iscen, iconfig, ispec, :] = self.species[specie].burden_per_emission
                self.pre_industrial_concentration_array[:, iscen, iconfig, ispec, :] = self.species[specie].pre_industrial_concentration
                self.natural_emissions_adjustment_array[:, iscen, iconfig, ispec, :] = self.species[specie].natural_emissions_adjustment
                self.radiative_efficiency_array[:, iscen, iconfig, ispec, :] = self.species[specie].radiative_efficiency
                self.g0_array[:, iscen, iconfig, ispec, :] = self.species[specie].g0
                self.g1_array[:, iscen, iconfig, ispec, :] = self.species[specie].g1
            if isinstance(self.species[specie], Halogen):
                self.fractional_release_array[:, iscen, iconfig, ispec, :] = self.species[specie].fractional_release
                self.br_atoms_array[:, iscen, iconfig, ispec, :] = self.species[specie].br_atoms
                self.cl_atoms_array[:, iscen, iconfig, ispec, :] = self.species[specie].cl_atoms
                
        self.cumulative_emissions_array = np.cumsum(self.emissions_array * self.time_deltas[:, None, None, None, None], axis=TIME_AXIS)
        self.alpha_lifetime_array = np.ones((n_timesteps, n_scenarios, n_configs, n_species, 1))
        self.airborne_emissions_array = np.zeros((n_timesteps, n_scenarios, n_configs, n_species, 1))
        
        # create temperature array of zeros if temperature not prescribed
        self.temperature_prescribed=True
        if not hasattr(self, 'temperature'):
            self.temperature = np.zeros((n_timesteps, n_temperature_boxes))
            self.temperature_prescribed=False
    
    def _fill_concentration(self):
        """After the emissions to concentrations step we want to put the concs into each GreenhouseGas"""
        iscen=0
        iconfig=0
        for ispec, specie in enumerate(self.species):
            if isinstance(self.species[specie], GreenhouseGas):
                self.species[specie].concentration = self.concentration_array[:, iscen, iconfig, ispec, 0]
                self.species[specie].cumulative_emissions = np.squeeze(self.cumulative_emissions_array[:, iscen, iconfig, ispec, 0])
                self.species[specie].airborne_fraction = self.airborne_emissions_array[:, iscen, iconfig, ispec, 0] / self.cumulative_emissions_array[:, iscen, iconfig, ispec, 0]
            if isinstance(self.species[specie], CH4):
                self.species[specie].effective_lifetime = self.alpha_lifetime_array[:, iscen, iconfig, ispec, 0] * self.lifetime_array[:, iscen, iconfig, ispec, 0]
    
    
    def _fill_forcing(self):
        """Add the forcing as an attribute to each Species"""
        for ispec, specie in enumerate(self.species):
            self.species[specie].forcing = self.forcing_array[:, 0, 0, ispec, 0]
            self.forcing = self.forcing_sum_array[:, 0, 0, 0, 0]
    
    
    def _perform_checks(self):
        if not hasattr(self, 'time'):
            raise TimeNotDefinedError("Time vector is not defined")
            
                
    def define_time(self, time):
        self.time = ensure_numeric_array(time)
        
        
    def prescribe_temperature(self, temperature):
        self.temperature = ensure_numeric_array(temperature)
        
    
    # this is the forward run - and should be generalised
    def run(self):
        
        # run initial sense checks on inputs.
        self._perform_checks()
        
        n_species = len(self.species)
        n_scenarios = 1  # just for now
        n_configs = 1 # just for now
        n_timesteps = len(self.time)
        self.time_deltas = _make_time_deltas(self.time)
        
        # initialise arrays. Using arrays makes things run quicker
        self._assign_indices()
        self._initialise_arrays(n_timesteps, n_scenarios, n_configs, n_species)
        gas_boxes = np.zeros((1, n_scenarios, n_configs, n_species, n_gas_boxes))
        temperature_boxes = np.zeros((1, n_scenarios, n_configs, 1, n_temperature_boxes))
        self.temperature_prescribed=False
#        if self.temperature_prescribed:
#            temperature_boxes = self.temperature[0, :]
        
        # initialise the energy balance model and get critical vectors
        # which itself needs to be run once per "config" and dimensioned correctly
        ebm = EnergyBalanceModel(
            ocean_heat_capacity=self.ocean_heat_capacity,
            ocean_heat_transfer=self.ocean_heat_transfer,
            deep_ocean_efficacy=self.deep_ocean_efficacy,
            stochastic_run=self.stochastic_run,
            sigma_eta=self.sigma_eta,
            sigma_xi=self.sigma_xi,
            gamma_autocorrelation=self.gamma_autocorrelation,
            seed=self.seed,
        )
        ebm_matrix_d = ebm.eb_matrix_d
        forcing_vector_d = ebm.forcing_vector_d
        
        for i_timestep in range(n_timesteps):
            # 1. ghg emissions to concentrations
            alpha_lifetime_array = calculate_alpha(
                self.cumulative_emissions_array[[i_timestep], ...],
                self.airborne_emissions_array[[i_timestep-1], ...],
                0,
                self.iirf_0_array,
                self.iirf_cumulative_array,
                self.iirf_temperature_array,
                self.iirf_airborne_array,
                self.g0_array,
                self.g1_array,
            )
            alpha_lifetime_array[np.isnan(alpha_lifetime_array)]=1  # CF4 seems to have an issue. Should we raise warning?
            self.concentration_array[[i_timestep], ...], gas_boxes, self.airborne_emissions_array[[i_timestep], ...] = step_concentration(
                self.emissions_array[[i_timestep], ...], 
                gas_boxes,
                self.airborne_emissions_array[[i_timestep-1], ...], 
                self.burden_per_emission_array,
                self.lifetime_array,
                alpha_lifetime=alpha_lifetime_array,
                pre_industrial_concentration=self.pre_industrial_concentration_array,
                timestep=self.time_deltas[i_timestep],
                partition_fraction=self.partition_fraction_array,
                natural_emissions_adjustment=self.natural_emissions_adjustment_array,
            )
            self.alpha_lifetime_array[[i_timestep], ...] = alpha_lifetime_array
            
            # 2. concentrations to emissions for ghg emissions:
            # TODO:
            
            # 3. Greenhouse gas concentrations to forcing
            self.forcing_array[[i_timestep], ...] = ghg(
                self.concentration_array[[i_timestep], ...],
                self.pre_industrial_concentration_array,
                self.tropospheric_adjustment_array, 
                self.radiative_efficiency_array,
                self.species_index_mapping
            )
            
            # 4. aerosol emissions to forcing
            # ozone here
            # contrails here
            # BC on snow here
            # strat water vapour here
            # land use here
            # solar here
            # volcanic here
        
            # 99. sum up all of the forcing calculated previously
            self.forcing_sum_array[[i_timestep], ...] = np.nansum(self.forcing_array[[i_timestep], ...], axis=SPECIES_AXIS, keepdims=True)
        
            # 100. run the energy balance model - if temperature not prescribed - updating temperature boxes
            temperature_boxes[0, 0, 0, 0, :] = self.temperature[i_timestep, :] + (1 - self.temperature_prescribed) * (
                ebm_matrix_d @ temperature_boxes[0, 0, 0, 0, :] + forcing_vector_d * self.forcing_sum_array[i_timestep, 0, 0, 0, 0]
            )
            self.temperature[i_timestep] = temperature_boxes[0, 0, 0, 0, :]
            
        self._fill_concentration()
        self._fill_forcing()


    def plot(self, var):
        pl.plot(np.arange(self.time), var)
# 

# # this one for the general forcing
# tropospheric_adjustment_array = np.ones((n_scenarios, n_species, 1, 1)) * np.nan
# radiative_efficiency_array = np.ones((n_scenarios, n_species, 1, 1)) * np.nan
# erf = np.ones((n_scenarios, n_species, n_timesteps, 1)) * np.nan

# # ozone
# fractional_release_array = np.zeros((1, n_species, 1, 1)) # gonna assume this doesn't change
# cl_atoms_array = np.zeros((1, n_species, 1, 1))
# br_atoms_array = np.zeros((1, n_species, 1, 1))
# o3_radiative_efficiency_array = np.zeros((1, n_species, 1, 1))

# for ispec, specie in enumerate(species_list):
#     # options pertaining to SLCFs
#     if specie in slcf_list:
#         pre_industrial_emissions_array[:, ispec, :, :] = pre_industrial_emissions[specie]
    
#     # options pertaining to GHGs
#     if specie in gas_list:

        
#     # options pertaining only to minor GHGs
#     if specie in minor_gas_list:
#         radiative_efficiency_array[:, ispec, :, :] = radiative_efficiency[specie]
        

#     # options pertaining to all species
#     tropospheric_adjustment_array[:, ispec, :, :] = tropospheric_adjustment[specie]
#     o3_radiative_efficiency_array[:, ispec, :, :] = o3_radiative_efficiency[specie]

# g0, g1 = calculate_g(lifetime_array, partition_fraction_array)
        
    # times
    # configs

In [ ]:
fair = FAIR()

In [ ]:
fair

In [ ]:
vars(fair)

In [ ]:
fair = FAIR(configs=(canesm5, noresm2, hadgem3, miroc6, giss))

In [ ]:
vars(fair)

In [ ]:
fair = FAIR(species=(co2, ch4, n2o, sulfur))

In [ ]:
fair

In [ ]:
vars(fair)

In [ ]:
for specie in ['CO2', 'CH4', 'N2O', 'Sulfur']:
    fair.species[specie].emissions = emissions['ssp245'][specie]

In [ ]:
fair.time=np.arange(1750.5,2501,1)

In [ ]:
fair.species

In [ ]:
isinstance(fair.species['CO2'], GreenhouseGas)

In [ ]:
%%time
fair.run()

In [ ]:
## %lprun -f ghg fair.run()

In [ ]:
# %lprun -f EnergyBalanceModel.forcing_vector_d scen.run()

In [ ]:
pl.plot(fair.species['CO2'].concentration)
pl.plot(fair.species['CH4'].concentration)
pl.plot(fair.species['N2O'].concentration)

In [ ]:
pl.plot(fair.forcing)

In [ ]:
pl.plot(fair.species['CO2'].forcing)
pl.plot(fair.species['CH4'].forcing)
pl.plot(fair.species['N2O'].forcing)

In [ ]:
pl.plot(fair.species['CO2'].airborne_fraction)
pl.plot(fair.species['CH4'].airborne_fraction)
pl.plot(fair.species['N2O'].airborne_fraction)

In [ ]:
pl.plot(fair.species['CO2'].cumulative_emissions)

In [ ]:
pl.plot(fair.temperature)